# Import Libraries

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import numpy as np
import pandas as pd
import tqdm
import yaml
import glob
import os
import gc

from datetime import datetime
from IPython.display import clear_output

In [ ]:
from src.components.feature_extraction import feature_extraction
from src.utils import get_root_directory

In [ ]:
# Get root directory of the project
root_dir = get_root_directory()

In [ ]:
# Maximize Column Display 
pd.set_option('display.max_colwidth', None)     # Display all content within each cell without truncation
pd.set_option('display.max_columns', None)      # Display all columns
pd.set_option('display.width', None)            # Display entire width of DataFrame is displayed

pd.set_option('display.max_rows', None)         # Display all rows

# Identify Detector Configuration Type

In [ ]:
# # Configurations
# signal_ids = [
#     "1285", "1290",
#     "1300", "1315", "1325", "1330", 
#     "1455", "1470", "1490",
#     "1500", "1555",
#     "1707", "1725", "1790", "1795", 
#     "1960",
#     "2055", 
#     "2485", 
#     "2665", 
#     # "D5I-3000"
# ]

In [ ]:
# df_detector_config_type = pd.DataFrame()

# for signal_id in tqdm.tqdm(signal_ids): 
#     df_config_id = pd.read_csv(f"../data/interim/atspm/fdot_d5/signal_config/{signal_id}.csv")

#     dict_approach_types = {
#         2: "major", 6: "major", 8: "minor", 4: "minor"
#     }
    
#     phase_nos = [phase_no for phase_no in dict_approach_types.keys() if phase_no in df_config_id["phaseNo"].unique()]
#     dict_approaches = {
#         df_config_id.query("phaseNo == @phase_no")["approach"].unique()[0]: phase_no for phase_no in phase_nos
#     }
    
#     dict_detector_config_type_id = {
#         "signalID": [], "approachType": [], "approachDir": [], "phaseNo": [], "laneType": [], 
#         "withStopbar": [], "withSetback": [], "withDualSetback": []
#     }
#     for approach in dict_approaches.keys():
#         df_config_id_approach = (
#             df_config_id[df_config_id["approach"] == approach]
#             .reset_index(drop=True)
#         )
#         approach_type = dict_approach_types[dict_approaches[approach]]
#         approach_dir = approach.split()[-1][:-1]
    
#         lane_nos = sorted(df_config_id_approach["laneNoFromLeft"].unique().tolist())
#         for lane_no in lane_nos:
#             df_config_id_lane = (
#                 df_config_id_approach[df_config_id_approach["laneNoFromLeft"] == lane_no]
#                 .reset_index(drop=True)
#             )
#             phase_nos = [int(phase_no) for phase_no in df_config_id_lane["phaseNo"].unique() if pd.notna(phase_no)]
#             phase_no = phase_nos[0] if phase_nos != [] else 0
            
#             lane_type = df_config_id_lane["laneType"].unique()[0]
            
#             distances = df_config_id_lane["stopBarDistance"].unique().tolist()
#             distances = [int(distance) for distance in distances if pd.notna(distance)]
                
#             if distances == []:
#                 with_stopbar = 0; with_setback = 0; with_dual_setback = 0    
#             else:
#                 with_stopbar = 1 if 0 in distances else 0
#                 distances = [distance for distance in distances if distance != 0]
                
#                 if distances == []:
#                     with_setback = 0; with_dual_setback = 0  
#                 else:
#                     with_setback = 1 if len(distances) == 1 else 0
#                     with_dual_setback = 1 if len(distances) > 1 else 0
                
    
#             dict_detector_config_type_id["signalID"].append(signal_id)
#             dict_detector_config_type_id["approachType"].append(approach_type)
#             dict_detector_config_type_id["approachDir"].append(approach_dir)
#             dict_detector_config_type_id["phaseNo"].append(phase_no)
#             dict_detector_config_type_id["laneType"].append(lane_type)
#             dict_detector_config_type_id["withStopbar"].append(with_stopbar)
#             dict_detector_config_type_id["withSetback"].append(with_setback)
#             dict_detector_config_type_id["withDualSetback"].append(with_dual_setback)
    
#     df_detector_config_type_id = pd.DataFrame(dict_detector_config_type_id)

#     dict_detector_config_type_combos = {
#         1: [0, 0, 0],
#         2: [1, 0, 0],
#         3: [0, 1, 0],
#         4: [1, 1, 0],
#         5: [0, 0, 1],
#         6: [1, 0, 1]
#     }

#     dict_phase_lane_combos = {
#         "Left": "LL",
#         "Through": "TT",
#         "Right": "TR"
#     } # TT: through phase - through lane, LL: left-turn phase - left turning lane
    
#     phase_lane_combos = []; detector_config_type_ids = []
#     for i in range(len(df_detector_config_type_id)):
#         phase_no = df_detector_config_type_id["phaseNo"][i]
#         lane_type = df_detector_config_type_id["laneType"][i]
    
#         if len(lane_type.split()) == 1:
#             phase_lane_combos.append(dict_phase_lane_combos[lane_type]) 
#         else:
#             if any(l in lane_type for l in ["Through", "Right"]) and (phase_no % 2 == 0):
#                 phase_lane_combos.append("TS")
#             else:
#                 phase_lane_combos.append("LS")
        
#         combo = df_detector_config_type_id.iloc[i, -3:].values.tolist()
    
#         detector_config_type_id = [k for k, v in dict_detector_config_type_combos.items() if v == combo][0]
#         detector_config_type_ids.append(detector_config_type_id)
    
#     df_detector_config_type_id["plCombo"] = phase_lane_combos
#     df_detector_config_type_id["type"] = detector_config_type_ids

#     df_detector_config_type_id = (
#         df_detector_config_type_id.pivot_table(
#             values="type", 
#             index=["signalID", "approachType", "approachDir"], 
#             columns="plCombo", 
#             aggfunc=lambda x: ", ".join(x.unique().astype(str))  # Convert unique values to comma-separated strings
#         )
#         .reset_index()
#     )
    
#     df_detector_config_type_id.columns.name = None  # Remove the column index name

#     df_detector_config_type = pd.concat([df_detector_config_type, df_detector_config_type_id], 
#                                         axis=0, ignore_index=True)

In [ ]:
# df_detector_config_type = (
#     df_detector_config_type[["signalID", "approachType", "approachDir", "TT", "TR", "TS", "LL", "LS"]]
# )

# df_detector_config_type.to_csv("../reports/detector_config_type.csv", 
#                                index=False)